In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 40)

from splink.duckdb.duckdb_linker import DuckDBLinker
import splink.duckdb.duckdb_comparison_library as cl

!date
!whoami
!uname -a
!pwd

Wed 09 Nov 2022 01:29:46 PM PST
ndbs
Linux int-slurm-sarchive-p0001 5.4.0-88-generic #99-Ubuntu SMP Thu Sep 23 17:29:00 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux
/mnt/share/homes/ndbs/notebooks


# Following Splink tutorial, referring also to Sam Shin's code

[Splink Tutorial](https://moj-analytical-services.github.io/splink/demos/00_Tutorial_Introduction.html)

[Sam Shin's RecordLinkage repo](https://github.com/SamShin/RecordLinkage)

[Nathaniel Blair-Stahn's fork of Sam's repo](https://github.com/NathanielBlairStahn/RecordLinkageTest)

# Define directories and load decennial census data

In [2]:
project_output_dir = '/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop'
output_subdir = 'results/special_last_names/florida/2022_10_14_10_49_32/population_table'
output_dir = f'{project_output_dir}/{output_subdir}'

!ls -l $output_dir

total 32224
-rw-rw-r-- 1 albrja IHME-Simulationscience 12622072 Oct 20 23:08 decennial_census.hdf
-rw-rw-r-- 1 albrja IHME-Simulationscience 20364830 Oct 14 11:56 state_table.hdf


In [3]:
decennial_census_path = f'{output_dir}/decennial_census.hdf'
with pd.HDFStore(decennial_census_path, 'r') as census_hdf:
    print(census_hdf.info())
    census_keys = census_hdf.keys()

<class 'pandas.io.pytables.HDFStore'>
File path: /mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/special_last_names/florida/2022_10_14_10_49_32/population_table/decennial_census.hdf
/year_2020            frame        (shape->[47444,10])
/year_2030            frame        (shape->[46440,10])
/year_2040            frame        (shape->[44626,10])


In [4]:
census_keys

['/year_2020', '/year_2030', '/year_2040']

In [5]:
# Keys exist for years 2020, 2030, 2040
years = [2020, 2030]
census = {year: pd.read_hdf(decennial_census_path, f'year_{year}') for year in years}

for year in years:
    print(year, census[year].shape)

2020 (47444, 10)
2030 (46440, 10)


# See which people appear in one census but not the other

In [6]:
census[2020].index.difference(census[2030].index)

Int64Index([   14,    27,    37,    47,    57,    67,    68,    71,    74,
              101,
            ...
            49928, 49932, 49948, 49955, 49962, 49964, 49969, 49982, 49984,
            49993],
           dtype='int64', length=7153)

In [7]:
census[2030].index.difference(census[2020].index)

Int64Index([   41,    60,    98,   130,   154,   157,   171,   250,   290,
              291,
            ...
            54270, 54271, 54272, 54273, 54274, 54275, 54276, 54277, 54278,
            54279],
           dtype='int64', length=6149)

# Filter to a smaller subset of records to test linking, and add a `'unique_id'` column for Splink

The individual people appear in the same order, unless someone is missing or a new person has been added, so we can filter to the first $n$ records to get similar but not identical lists.

In [8]:
max_index = 1000

def prep_for_linking(df, max_index):
    return (
        df.loc[:max_index]
        .assign(
            unique_id=lambda df: df.index, # Splink requires a column called 'unique_id'
            age = lambda df: df['age'].floordiv(1).astype(int), # Replace age with floor(age)
        ) 
    )

subcensus = {year: prep_for_linking(df, max_index) for year, df in census.items()}

{year: df.shape for year, df in subcensus.items()}

{2020: (960, 11), 2030: (865, 11)}

In [9]:
subcensus[2020]

,first_name,last_name,age,date_of_birth,address,zipcode,relation_to_household_head,sex,race_ethnicity,middle_initial,unique_id
0,Margaret,Clark,68,1951-07-27,"1344 winoka rd brooksville, fl",34601,Reference person,Female,Black,J,0
1,Jeffrey,Littlejohn,52,1967-05-03,"927 23rd st clearwater, fl",34698,Reference person,Male,Black,V,1
2,Briana,Jackson,13,2006-09-07,"927 23rd st clearwater, fl",34698,Biological child,Female,Black,A,2
3,Benjamin,Cox,21,1998-10-21,"927 23rd st clearwater, fl",34698,Stepchild,Male,Black,D,3
4,Willie,Tucker,72,1947-10-09,"8904 167th place fleming island, fl",32003,Reference person,Male,White,J,4
...,...,...,...,...,...,...,...,...,...,...,...
996,Cassandra,Stevens,56,1963-08-14,"4475 13th st spring hill, fl",34606,Reference person,Female,White,S,996
997,James,Hansen,64,1955-08-13,"4475 13th st spring hill, fl",34606,Opp-sex spouse,Male,White,R,997
998,Deanna,Miller,28,1991-05-03,"4475 13th st spring hill, fl",34606,Biological child,Female,White,R,998
999,Denzel,Heaney,26,1993-05-06,"4475 13th st spring hill, fl",34606,Biological child,Male,White,N,999


In [10]:
subcensus[2030]

,first_name,last_name,age,date_of_birth,address,zipcode,relation_to_household_head,sex,race_ethnicity,middle_initial,unique_id
0,Margaret,Clark,70,1951-07-27,"1344 winoka rd brooksville, fl",34601,Reference person,Female,Black,J,0
1,Jeffrey,Littlejohn,53,1967-05-03,"1400 miller ave unincorporated, fl",33428,Reference person,Male,Black,V,1
2,Briana,Jackson,23,2006-09-07,"mirasol drive kendall, fl",33157,Other nonrelative,Female,Black,A,2
3,Benjamin,Cox,31,1998-10-21,"272 hickey road palm coast, fl",32164,Other nonrelative,Male,Black,D,3
4,Willie,Tucker,78,1947-10-09,"8904 167th place fleming island, fl",32003,Reference person,Male,White,J,4
...,...,...,...,...,...,...,...,...,...,...,...
996,Cassandra,Stevens,66,1963-08-14,"966 s 730 w fort lauderdale, fl",33317,Reference person,Female,White,S,996
997,James,Hansen,74,1955-08-13,"966 s 730 w fort lauderdale, fl",33317,Opp-sex spouse,Male,White,R,997
998,Deanna,Miller,38,1991-05-03,"966 s 730 w fort lauderdale, fl",33317,Biological child,Female,White,R,998
999,Denzel,Heaney,36,1993-05-06,"15509 vail cut off rd se jacksonville, fl",32254,Other nonrelative,Male,White,N,999


In [11]:
subcensus[2020].index.difference(subcensus[2030].index)

Int64Index([ 14,  27,  37,  47,  57,  67,  68,  71,  74, 101,
            ...
            917, 923, 925, 926, 940, 947, 951, 960, 978, 982],
           dtype='int64', length=132)

In [12]:
subcensus[2030].index.difference(subcensus[2020].index)

Int64Index([ 41,  60,  98, 130, 154, 157, 171, 250, 290, 291, 327, 373, 408,
            414, 428, 431, 441, 444, 539, 550, 552, 570, 588, 620, 647, 690,
            736, 748, 753, 787, 836, 856, 862, 882, 886, 910, 922],
           dtype='int64')

# Look at how addresses change from 2020 to 2030

In [13]:
columns = ['first_name', 'last_name', 'address', 'zipcode']
df1 = subcensus[2020].loc[:,columns]
df2 = subcensus[2030].loc[:,columns]
df1.reindex(df2.index).compare(df2)

first_name       last_name        \
           self other      self other   
1           NaN   NaN       NaN   NaN   
2           NaN   NaN       NaN   NaN   
3           NaN   NaN       NaN   NaN   
6           NaN   NaN       NaN   NaN   
9           NaN   NaN       NaN   NaN   
...         ...   ...       ...   ...   
996         NaN   NaN       NaN   NaN   
997         NaN   NaN       NaN   NaN   
998         NaN   NaN       NaN   NaN   
999         NaN   NaN       NaN   NaN   
1000        NaN   NaN       NaN   NaN   

                                        address  \
                                           self   
1                   927 23rd st  clearwater, fl   
2                   927 23rd st  clearwater, fl   
3                   927 23rd st  clearwater, fl   
6     3478 claude douglas cir  jacksonville, fl   
9                      94 perry rd  orlando, fl   
...                                         ...   
996               4475 13th st  spring hill, fl   
997               4475 13th st  spring hill, fl   
998               4475 13th st  spring hill, fl   
999               4475 13th st  spring hill, fl   
1000              4475 13th st  spring hill, fl   

                                                 zipcode         
                                           other    self  other  
1            1400 miller ave  unincorporated, fl   34698  33428  
2                     mirasol drive  kendall, fl   34698  33157  
3                272 hickey road  palm coast, fl   34698  32164  
6               1838 kelton ln  jacksonville, fl   32218  32224  
9             2795 north eagle rd  florahome, fl   32832  32140  
...                                          ...     ...    ...  
996             966 s 730 w  fort lauderdale, fl   34606  33317  
997             966 s 730 w  fort lauderdale, fl   34606  33317  
998             966 s 730 w  fort lauderdale, fl   34606  33317  
999   15509 vail cut off rd se  jacksonville, fl   34606  32254  
1000             1898 woodside dr  kissimmee, fl   34606  34743  

[414 rows x 8 columns]

In [14]:
df2.reindex(df1.index).compare(df1)

first_name       last_name        \
           self other      self other   
1           NaN   NaN       NaN   NaN   
2           NaN   NaN       NaN   NaN   
3           NaN   NaN       NaN   NaN   
6           NaN   NaN       NaN   NaN   
9           NaN   NaN       NaN   NaN   
...         ...   ...       ...   ...   
996         NaN   NaN       NaN   NaN   
997         NaN   NaN       NaN   NaN   
998         NaN   NaN       NaN   NaN   
999         NaN   NaN       NaN   NaN   
1000        NaN   NaN       NaN   NaN   

                                         address  \
                                            self   
1            1400 miller ave  unincorporated, fl   
2                     mirasol drive  kendall, fl   
3                272 hickey road  palm coast, fl   
6               1838 kelton ln  jacksonville, fl   
9             2795 north eagle rd  florahome, fl   
...                                          ...   
996             966 s 730 w  fort lauderdale, fl   
997             966 s 730 w  fort lauderdale, fl   
998             966 s 730 w  fort lauderdale, fl   
999   15509 vail cut off rd se  jacksonville, fl   
1000             1898 woodside dr  kissimmee, fl   

                                                zipcode         
                                          other    self  other  
1                   927 23rd st  clearwater, fl   33428  34698  
2                   927 23rd st  clearwater, fl   33157  34698  
3                   927 23rd st  clearwater, fl   32164  34698  
6     3478 claude douglas cir  jacksonville, fl   32224  32218  
9                      94 perry rd  orlando, fl   32140  32832  
...                                         ...     ...    ...  
996               4475 13th st  spring hill, fl   33317  34606  
997               4475 13th st  spring hill, fl   33317  34606  
998               4475 13th st  spring hill, fl   33317  34606  
999               4475 13th st  spring hill, fl   32254  34606  
1000              4475 13th st  spring hill, fl   34743  34606  

[509 rows x 8 columns]

# Create a linker and profile some columns

[Exploratory Analysis in Splink](https://moj-analytical-services.github.io/splink/demos/01_Exploratory_analysis.html)

In [59]:
# Minimal settings needed to call .profile_columns()
initial_settings = {'link_type': 'link_only'}
linker = DuckDBLinker([subcensus[2020], subcensus[2030]], initial_settings)

In [60]:
linker.profile_columns(['first_name', 'last_name'])

In [61]:
linker.profile_columns(['address', 'zipcode'])

# Indexing/Blocking

[Blocking in Splink](https://moj-analytical-services.github.io/splink/demos/02_Blocking.html)

From the documentation:

> ### Devising effective blocking rules
>
> The aims of your blocking rules are twofold: 1. Eliminate enough non-matching comparison pairs so your record linkage job is small enough to compute 2. Eliminate as few truly matching pairs as possible (ideally none)
>
> It is usually impossible to find a single blocking rule which achieves both aims, so we recommend using multiple blocking rules.
>
> When we specify multiple blocking rules, Splink will generate all comparison pairs that meet any one of the rules.
>
> ### Blocking rules in Splink
>
> In Splink, blocking rules are specified as SQL expressions.
>
> For example, to generate the subset of record comparisons where the first name matches, we can specify the following blocking rule:
>
> `l.first_name = r.first_name`
>
> Since blocking rules are SQL expressions, they can be arbitrarily complex. For example, you could create record comparisons where the initial of the first name and the surname match with the following rule:
>
> `substr(l.first_name, 1,1) = substr(r.first_name, 1,1) and l.surname = r.surname`

In [62]:
blocking_rules = [
#     'l.zipcode = r.zipcode',
    'l.first_name = r.first_name and l.last_name = r.last_name',
    'l.sex = r.sex and l.zipcode = r.zipcode',
    (
        'l.sex = r.sex'
        ' and abs(round(r.age) - (round(l.age)+10)) <= 1'
#         ' and substr(l.first_name, 1,1) = substr(r.first_name, 1,1)'
        ' and substr(l.last_name, 1,1) = substr(r.last_name, 1,1)'
    ), # Age, sex, and initials of first and last name
]

for rule in blocking_rules:
    count = linker.count_num_comparisons_from_blocking_rule(rule)
    print(f"Number of comparisons generated by '{rule}': {count:,.0f}")

linker.cumulative_num_comparisons_from_blocking_rules_chart(blocking_rules)

Number of comparisons generated by 'l.first_name = r.first_name and l.last_name = r.last_name': 830
Number of comparisons generated by 'l.sex = r.sex and l.zipcode = r.zipcode': 1,587
Number of comparisons generated by 'l.sex = r.sex and abs(round(r.age) - (round(l.age)+10)) <= 1 and substr(l.last_name, 1,1) = substr(r.last_name, 1,1)': 1,724


# Make a settings dictionary and do a comparison

[Specifying and estimating a linkage model in Splink](https://moj-analytical-services.github.io/splink/demos/03_Estimating_model_parameters.html)


From [Comparison](https://moj-analytical-services.github.io/splink/comparison.html) documentation page:

> As far as possible, Comparisons should be configured to satisfy the assumption of independece conditional on the true match status, a key assumption of the Fellegi Sunter probabilistic linkage model. This would be broken, for example, if a model contained one Comparison for city, and another for postcode. Instead, in this example, a single comparison should be modelled, which may to capture similarity taking account of both the city and postcode field.


https://moj-analytical-services.github.io/splink/topic_guides/customising_comparisons.html

https://moj-analytical-services.github.io/splink/settingseditor/editor.html

https://moj-analytical-services.github.io/splink/settings_dict_guide.html#probability_two_random_records_match


From Tutorial [03 Estimating model parameters](https://moj-analytical-services.github.io/splink/demos/03_Estimating_model_parameters.html) documentation:

> ### Estimation of `probability_two_random_records_match`
>
> In some cases, the `probability_two_random_records_match` will be known. For example, if you are linking two tables of 10,000 records and expect a one-to-one match, then you should set this value to 1/10_000 [in your settings](https://moj-analytical-services.github.io/splink/settings_dict_guide.html#probability_two_random_records_match) instead of estimating it.
> 
> More generally, this parameter is unknown and needs to be estimated.
> 
> It can be estimated accurately enough for most purposes by combining a series of deterministic matching rules and a guess of the recall corresponding to those rules. For further details of the rationale behind this appraoch see [here](https://github.com/moj-analytical-services/splink/issues/462#issuecomment-1227027995).
>
> In this example, I guess that the following deterministic matching rules have a recall of about 70%:
>
>    ```python
>    deterministic_rules = [
>        "l.first_name = r.first_name and levenshtein(r.dob, l.dob) <= 1",
>        "l.surname = r.surname and levenshtein(r.dob, l.dob) <= 1",
>        "l.first_name = r.first_name and levenshtein(r.surname, l.surname) <= 2",
>        "l.email = r.email"
>    ]
>
>    linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.7)
>    ```

**Question:** Why does this work, and how are you supposed to estimate the recall?

The github issue in referred in the documentation has partial answers, but I think does not explain things sufficiently for someone who doesn't understand the F-S algorithm well: https://github.com/moj-analytical-services/splink/issues/462#issuecomment-1227027995

**Question:** What if you don't make a good guess? Will the EM algorithm perform badly?

From the [settings dict guide documentation on `probability_two_random_records_match`](https://moj-analytical-services.github.io/splink/settings_dict_guide.html#probability_two_random_records_match):

> If you estimate parameters using expectation maximisation (EM), this provides an initial value (prior) from which the EM algorithm will start iterating. EM will then estimate the true value of this parameter.
>
> **Default value**: 0.0001
>
> **Examples**: [1e-05, 0.006]

In my initial test using the records with index under 1000 from the 2020 and 2030 censuses (960 and 865 records, respectively), the true probability of a match is

$$
\frac{\text{number of true matches}}{\text{number of pairs}} = 
\frac{828}{960 \cdot 865} = 0.0009971\ldots \approx 0.001.
$$

Note that if we didn't look at the actual number of records in each data set, and instead just assumed that each of them had the full 1000 records originally specified, then this true probability is quite close to the value we would get using the one-to-one assumption as mentioned in the Splink documentation above, which would be

$$
\frac{\text{number of true matches}}{\text{number of pairs}} =
\frac{1000}{1000 \cdot 1000} = 0.001.
$$

I'm not sure whether there's any good reason for this besides the fact that both datasets still have approximately 1000 records, and the 1:1 assumption should be close to true.

In [63]:
s = pd.Series([3.45, 2.984])
s

0    3.450
1    2.984
dtype: float64

In [64]:
s.floordiv(1)

0    3.0
1    2.0
dtype: float64

In [65]:
def true_probability_two_random_records_match(df1, df2):
    size_intersection = len(df1.index.intersection(df2.index))
    print(size_intersection)
    return size_intersection / (len(df1) * len(df2))

true_probability_two_random_records_match(subcensus[2020], subcensus[2030])

828


0.0009971098265895953

In [66]:
1/1000 # Assume both datasets had the original specified length of 1000 and matching is 1:1

0.001

In [67]:
828 / 1000**2 # Compare with true number of matches / inflated denominator

0.000828

In [68]:
print(subcensus[2020].shape, subcensus[2030].shape)

(960, 11) (865, 11)


In [69]:
1/(960*865)**(1/2) # Assume 1:1 match with same of number of records equal to geometric mean

0.0010973782032653676

In [25]:
1/960 # Assume 1:1 match of all records from smaller dataset

0.0010416666666666667

In [70]:
# # Example from https://moj-analytical-services.github.io/splink/topic_guides/customising_comparisons.html
#
# comparison_first_name = {
#     "output_column_name": "first_name",
#     "comparison_description": "First name jaro dmeta",
#     "comparison_levels": [
#         {
#             "sql_condition": "first_name_l IS NULL OR first_name_r IS NULL",
#             "label_for_charts": "Null",
#             "is_null_level": True,
#         },
#         {
#             "sql_condition": "first_name_l = first_name_r",
#             "label_for_charts": "Exact match",
#             "tf_adjustment_column": "first_name",
#             "tf_adjustment_weight": 1.0,
#             "tf_minimum_u_value": 0.001,
#         },
#         {
#             "sql_condition": "dmeta_first_name_l = dmeta_first_name_r",
#             "label_for_charts": "Exact match",
#             "tf_adjustment_column": "dmeta_first_name",
#             "tf_adjustment_weight": 1.0,
#         },
#         {
#             "sql_condition": "jaro_winkler_sim(first_name_l, first_name_r) > 0.8",
#             "label_for_charts": "Exact match",
#             "tf_adjustment_column": "first_name",
#             "tf_adjustment_weight": 0.5,
#             "tf_minimum_u_value": 0.001,
#         },
#         {"sql_condition": "ELSE", "label_for_charts": "All other comparisons"},
#     ],

# }

age_comparison = {
    "output_column_name": "age",
    "comparison_description": "Age difference approximately 10",
    "comparison_levels": [
        {
            "sql_condition": "age_l IS NULL OR age_r IS NULL",
            "label_for_charts": "Null",
            "is_null_level": True,
        },
        {
            "sql_condition": "round(age_r) - round(age_l) = 10",
            "label_for_charts": "Exact match",
        },
        {
            "sql_condition": "abs(round(age_r) - (round(age_l)+10)) = 1",
            "label_for_charts": "Age within 1 year",
        },
        {"sql_condition": "ELSE", "label_for_charts": "Age difference >= 2 years"},
    ],
}

In [71]:
settings = {
    "link_type": "link_only",
    "comparisons": [
        cl.levenshtein_at_thresholds("first_name"),
        cl.levenshtein_at_thresholds("last_name"),
#         cl.levenshtein_at_thresholds("date_of_birth", 1),
        cl.exact_match("sex"),
        age_comparison,
#         cl.levenshtein_at_thresholds("zipcode"),
    ],
    "blocking_rules_to_generate_predictions": blocking_rules,
    "probability_two_random_records_match":
        true_probability_two_random_records_match(subcensus[2020], subcensus[2030]),
    "retain_matching_columns": True,
    "retain_intermediate_calculation_columns": True,
}

linker.initialise_settings(settings)

828


## Estimate u probabilities

https://moj-analytical-services.github.io/splink/demos/03_Estimating_model_parameters.html#estimation-of-u-probabilities

In [72]:
%%time
linker.estimate_u_using_random_sampling(target_rows=1e6)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - last_name (no m values are trained).
    - sex (no m values are trained).
    - age (no m values are trained).


CPU times: user 1.6 s, sys: 2.75 ms, total: 1.6 s
Wall time: 837 ms


## Estimate m probabilities

https://moj-analytical-services.github.io/splink/demos/03_Estimating_model_parameters.html#estimation-of-m-probabilities

Difference between blocking rules for prediction (once the EM algorithm has estimated parameters) vs. estimation (training of EM algorithm):

https://moj-analytical-services.github.io/splink/topic_guides/blocking_rules.html

https://www.robinlinacre.com/comparing_splink_models_unsupervised/

In [73]:
training_blocking_rules = ["l.last_name = r.last_name", 'l.zipcode = r.zipcode', 'l.age + 10 = r.age']
for rule in training_blocking_rules:
    count = linker.count_num_comparisons_from_blocking_rule(rule)
    print(f"Number of comparisons generated by '{rule}': {count:,.0f}")

linker.cumulative_num_comparisons_from_blocking_rules_chart(training_blocking_rules)
# training_session_fname_sname = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)


Number of comparisons generated by 'l.last_name = r.last_name': 1,224
Number of comparisons generated by 'l.zipcode = r.zipcode': 2,835
Number of comparisons generated by 'l.age + 10 = r.age': 10,468


In [74]:
%%time
training_sessions = {}
for rule in training_blocking_rules:
    training_sessions[rule] = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rules[1])
    
    


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.zipcode = r.zipcode

Parameter estimates will be made for the following comparison(s):
    - first_name
    - last_name
    - sex
    - age

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 


Level levenshtein <= 1 on comparison last_name not observed in dataset, unable to train m value
Iteration 1: Largest change in params was 0.152 in probability_two_random_records_match

Level levenshtein <= 1 on comparison last_name not observed in dataset, unable to train m value
Iteration 2: Largest change in params was 0.0327 in the m_probability of age, level `Age difference >= 2 years`

Level levenshtein <= 1 on comparison last_name not observed in dataset, unable to train m value
Iteration 3: Largest change in params was 0.00139 in the m_probability of last_name, level `All other comparisons`

Level levenshtein <= 1 on compariso

CPU times: user 1.14 s, sys: 37.8 ms, total: 1.18 s
Wall time: 1.11 s


## Make an iterator and look at the three training sessions

These all look very similar. I'm not sure how to tell what's going on, e.g. whether the subsequent blocking rules didin't make much difference, as it appears below, or whether the original training session got updated when the later sessions were run.

In [75]:
rule_session = iter(training_sessions.items())

In [76]:
rule, session = next(rule_session)
print(rule)
session.m_u_values_interactive_history_chart()

l.last_name = r.last_name


In [77]:
rule, session = next(rule_session)
print(rule)
session.m_u_values_interactive_history_chart()

l.zipcode = r.zipcode


In [78]:
rule, session = next(rule_session)
print(rule)
session.m_u_values_interactive_history_chart()

l.age + 10 = r.age


# Visualize model parameters

https://moj-analytical-services.github.io/splink/demos/03_Estimating_model_parameters.html#visualising-model-parameters

In [80]:
linker.match_weights_chart()

In [81]:
linker.m_u_parameters_chart()

# Look for unlinkable records

https://moj-analytical-services.github.io/splink/demos/03_Estimating_model_parameters.html#detecting-unlinkable-records

The graph shows nothing. Does that mean there are none?

In [82]:
linker.unlinkables_chart()

# Saving the model parameters and reloading them to do prediction/linking

[Saving the model](https://moj-analytical-services.github.io/splink/demos/03_Estimating_model_parameters.html#saving-the-model):

```python
linker.save_settings_to_json("./demo_settings/saved_model_from_demo.json", overwrite=True)
```

[Loading saved model](https://moj-analytical-services.github.io/splink/demos/04_Predicting_results.html#load-estimated-model-from-previous-tutorial):

```python
linker = DuckDBLinker(df) # The demo was for de-duplication, so only one df
linker.load_settings_from_json("./demo_settings/saved_model_from_demo.json")
```

In [85]:
linker.save_settings_to_json(
    "./splink_test_data/saved_model_from_census_test.json",
    overwrite=True
)

# Save input dataframes since they will also be needed to reload the model

In [86]:
subcensus.keys()

dict_keys([2020, 2030])

In [91]:
for year, df in subcensus.items():
    print(df.shape)
    df.to_csv(f"./splink_test_data/census_{year}_test_sample.csv")

(960, 11)
(865, 11)


In [90]:
!ls -l splink_test_data/

total 220
-rw-rw-r-- 1 ndbs Domain Users 104935 Nov 10 16:33 census_2020_test_sample.csv
-rw-rw-r-- 1 ndbs Domain Users  95387 Nov 10 16:33 census_2030_test_sample.csv
-rw-rw-r-- 1 ndbs Domain Users   5771 Nov  9 17:31 saved_model_from_census_test.json


# Predicting which records match

[04 Predicting results](https://moj-analytical-services.github.io/splink/demos/04_Predicting_results.html)
(Predicting which records match)

> We use `linker.predict()` to run the model.
>
> Under the hood this will:
>
> Generate all pairwise record comparisons that match at least one of the blocking_rules_to_generate_predictions
>
> Use the rules specified in the Comparisons to evaluate the similarity of the input data
>
> Use the estimated match weights, applying term frequency adjustments where requested to produce the final match_weight and match_probability scores
>
> Optionally, a `threshold_match_probability` or `threshold_match_weight` can be provided, which will drop any row where the predicted score is below the threshold.

In [83]:
%%time
df_predictions = linker.predict(threshold_match_probability=0.2)
df_predictions.as_pandas_dataframe(limit=5)


 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'last_name':
    m values not fully trained


CPU times: user 87.7 ms, sys: 20.3 ms, total: 108 ms
Wall time: 91.6 ms


,match_weight,match_probability,source_dataset_l,unique_id_l,source_dataset_r,unique_id_r,first_name_l,first_name_r,gamma_first_name,bf_first_name,...,sex_r,gamma_sex,bf_sex,age_l,age_r,gamma_age,bf_age,zipcode_l,zipcode_r,match_key
0,4.323437,0.952428,_a,0,_b,0,Margaret,Margaret,3,130.301965,...,Female,1,2.0,68,70,0,0.151487,34601,34601,0
1,4.323437,0.952428,_a,1,_b,1,Jeffrey,Jeffrey,3,130.301965,...,Male,1,2.0,52,53,0,0.151487,34698,33428,0
2,13.212943,0.999895,_a,2,_b,2,Briana,Briana,3,130.301965,...,Female,1,2.0,13,23,2,71.842787,34698,33157,0
3,13.212943,0.999895,_a,3,_b,3,Benjamin,Benjamin,3,130.301965,...,Male,1,2.0,21,31,2,71.842787,34698,32164,0
4,4.323437,0.952428,_a,4,_b,4,Willie,Willie,3,130.301965,...,Male,1,2.0,72,78,0,0.151487,32003,32003,0


In [84]:
df_predictions.as_pandas_dataframe().shape

(832, 25)